# AAI30001 Small Project
#### **Group: SP_8**
 - Chua Chen Yi (2302822)
 - Wong Jun Jai (2302765)

# Methodology

#### Our proposed method of improving accuracy over the baseline score is as follows:
### 1. Identify a pre-trained model to be our baseline for fine-tuning
- Before starting our search, we manually re-created the same testing environment as the sample code. This included splitting the dataset into training, validation and testing identically to the sample. In addition, the use of the TestUA for determining overall performance was use
    - *Minor note: We have noticed the way the dataset is split for testing is not the same as how it is described in the text. For example, the test set should only contain files from 'Ses01F' and only from the female speaker. However, checking the dataset class showed that this is not true.*
- With a simple evaluation pipeline in placee, we randomly choose 10 different models publically availble from hugging face, and proceeded to score them. They scored a range from 0.60-0.80 on the test dataset.
- We decided to use the same base model: **"facebook/wav2vec2-base"** with the goal of fine tuning according to the rules and achieve at least 0.70
### 2. Perform Data Augmentation On Dataset
- #### Augmentation 1
    - Details
- #### Augmentation 2
    - Details
### 3. Identify Strengths & Weakness of Pre-Trained Model
- We started by training the model for a few epochs to get a general idea of how it performs. A sample confusion matrix is shown here:

             A   H    N   S
        A  103   8   35   1
        H    8  79   36   9
        N    2   7  109  53 <- Highest Error
        S    1   8   10  59
        
    In general, the model is able to differentiate angry and happy emotions with a high degree of accuracy. However, the model is not good at differentiaing between neutral and sad emotions. Its greatest weakness is predicting a neutral emotion as a sad one
                
### 4. Use Secondary Model With Text Embeddings
- We have decided not to use extracted text embeddings as a feature of our first model, but instead have a completely seperate model extract and perform sentiment analysis on the text. The final prediction will be a combination of both models.
    - This method allows us to:
        1. Manage our time better as work can be done to improve the performance of both indepenently.
        2. Change our base model if we find a better one.
        3. Choose more sophisticated speech-to-text and sentiment analysis models
### 5. Develop Algorithm To Merge Predictions
- Our final implementation consist of getting the confidence for each label using softmax in addition to its original prediction. The predictions for the 2nd model is then merged into a single CSV file. An gridsearch-like function will identify the optimal parameters *(highlighted in **Bold**)*.
- All models tested have improved scores. In general, models with <0.65 score will see a boost of 2-4%, while models >0.65 score will gain 0-2%. 
    ### Merge Algorithm
    - The algorithm that determines the final predictions is a combination of 3 different strategies:
        1. **Merging Strategy:** When to rely on the 2nd model?
        2. **Prediction Strategy:** How to rely on the 2nd model?
        3. **Mapping Strategy:** How to map sentiment to emotions?
    ### 5A. Merging Strategy
    We have identified 2 possible metrics to decide when to rely on the 2nd model
    - **Entrophy Threshold**
    We apply a calculate the entrophy based on the 4 confidence scores, following the logic that a lower overall entrophy will mean the model is most confident in its prediction. We will refer to the 2nd model when the entrophy is above the ***entrophy_threshold***.
    - **Argmax Threshold**
    We apply a simple argmax on the 4 confidence scores. If the value is below the ***argmax_threshold***, we will refer to the 2nd model for the final prediction
    ### 5B. Prediction Strategy
    We have identified 3 possible metrics to decide how to rely on the 2nd model
    - **Default**
    Prefer prediction of 2nd model in all situations
    - **Ignore**
    We identified the original model is very good at angry and happy emotions. So we always prefer the orignal model's predictions if it detects angry or happy
    - **Ignore When Match**
    If both models agree on the same prediction, we ignore however low the confidence is and assume is correct
    ### 5C. Mapping Strategy
    Because our sentiment analysis outputs 3 classes, while we have 4 emotions, we ill need to map 1 class to 2 emotions. This corresponds to the 'Negative' sentiment being mapped to either 'Angry' or 'Sad'. We have implemented the following methods:
    - **Simple Mapping**
    We decide on a ***sentiment_threshold*** value, where anything above is 'Sad'. This can also ***fliped*** around, ie. anything above is 'Angry' as there is no defined way to map the negative sentiment
    - **Reference Mapping**
    When a negative sentiment needs to be mapped, the confidence of 'Sad' and 'Angry' from the original model is looked up, and Argmax is used to return the most likely emotion.
    
    
    
    
    
